### Skin Disease Classification Using Deep Learning Technique

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.applications.inception_v3 import InceptionV3
import matplotlib.pyplot as plt


- Set directory for train & test dataset
- Define size of image class from number of folder

In [ ]:

train_dir = 'data/train'
test_dir = 'data/test'

class_size = len(os.listdir(train_dir))

- Define the image resolation for model(size)

In [ ]:
height = 300
widht = 300

- Calculate number of image data for both train & test

In [ ]:
train_size = 0
test_size = 0
for fd in os.listdir(train_dir):
    train_size += len(os.listdir(os.path.join(train_dir, fd)))
for fd in os.listdir(test_dir):
    test_size += len(os.listdir(os.path.join(test_dir, fd)))



- Defining & Loading weight for pre-trained model (InceptionV3)
- Marking model trainable propertry to false to prevent retrain

In [ ]:
pre_trained_model = InceptionV3(input_shape = (widht, height, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

for layer in pre_trained_model.layers:
  layer.trainable = False
  
pre_trained_model.summary()



### Adding custom layer to the pre-trained model

In [ ]:
last_layer = pre_trained_model.get_layer('mixed8')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

x = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(last_output)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.2)(x)
x = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.4)(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu', kernel_initializer='he_uniform')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(class_size, activation='softmax')(x)

model = Model( pre_trained_model.input, x)    

- Configuring callback for saving model weight during training to retain result & avoid retraining

In [ ]:
checkpoint_path = "training_dermnet/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

- reload model weight from previous checkpoint

In [ ]:
model.load_weights(checkpoint_path)

### Initializing data augmentation
- Generate new images from existing images by resizing or
scaling, cropping, zooming and flipping the images.
- normalize the images in range (0-1)

In [ ]:
# Creating Train, Test DataGenerator
train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range=90,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
    
test_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range=90,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True, 
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size=20,
    class_mode='categorical',
    target_size=(widht, height))


test_generator = train_datagen.flow_from_directory(
    test_dir,
    batch_size=20,
    class_mode='categorical',
    target_size=(widht, height))

- Defining model loss function as Categorical Crossentropy 
- Stochastic Gradient Descent as optimizer
- learning rate to 0.01
- momentum to 0.9

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(learning_rate=0.0001, momentum=0.9),
              metrics='accuracy')

### Train Model 

In [ ]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    steps_per_epoch=train_size/20, #step_per_epoch * batch_size = total_images
    epochs=500,
    validation_steps=test_size/20,
    callbacks=[cp_callback],
    verbose=1
)

- Draw accuracy & loss graph to visualize the progress

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title ('Training and validation accuracy')
plt.figure()

plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )